In [8]:
#using Pkg
#Pkg.add("Profile")

In [11]:
using DelimitedFiles
using LinearAlgebra
using Optim
using Distributions
using Test
using BenchmarkTools
using Random
using CSV
using DataFrames
using Distributed
using Plots
using Profile

In [2]:
pwd()

"/Users/FredYu/Documents/GitHub/BulkLMM.jl/analysis"

In [3]:
## Include the source code of BulkLMM to be tested:
include("../src/scan.jl");
include("../src/lmm.jl");
include("../src/wls.jl");
include("../src/util.jl");
include("../src/kinship.jl");
include("../src/readData.jl");

## Also include the helper functions for writing tests:
include("../test/testHelper.jl");

In [4]:
## Read in BXD data:
pheno_file = "../data/bxdData/BXDtraits.csv"
pheno = readBXDpheno(pheno_file);
geno_file = "../data/bxdData/BXDgeno_prob.csv"
geno = readGenoProb_ExcludeComplements(geno_file);

kinship = calcKinship(geno); # calculate kinship matrix from genotype data

In [5]:
nperms = 1024; # number of permutated copies required;
m = size(pheno, 2);
p = size(geno, 2); # number of markers

In [6]:
## Consider the 7919-th trait
pheno_y = reshape(pheno[:, 7919], :, 1);

## Do the same for many traits (first 1000):

In [7]:
# Function that extracts the row maximums for a matrix:
function getMaxLod_manyTraits(pheno::Array{Float64, 2}, geno::Array{Float64, 2}, kinship::Array{Float64, 2};
                              ntraits::Int64 = 1000, nperms::Int64 = 1024, original::Bool = true)
    
    if original
        nrows = nperms+1
    else
        nrows = nperms
    end
    
    maxLods_manyTraits = Matrix{Float64}(undef, nrows, ntraits);
    
    for k in 1:ntraits
        
        pheno_k = reshape(pheno[:, k], :, 1);
        results = scan_perms(pheno_k, geno, kinship; nperms = nperms)
        
        maxLods_manyTraits[:, k] = getMaxLod(results)
    end
    
    return maxLods_manyTraits; 
end


function getMaxLod(results::Array{Float64, 2})
    
    return mapslices(x -> maximum(x), results; dims = 2);
    
end

getMaxLod (generic function with 1 method)

In [8]:
@benchmark scan_perms(pheno_y, geno, kinship; nperms = 1024)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 6.713 s (11.08% GC) to evaluate,
 with a memory estimate of 13.63 GiB, over 194772 allocations.

In [9]:
function myfunc(x)
    return x+1;
end

@profile myfunc(2)

3

In [10]:
Profile.print()

Overhead ╎ [+additional indent] Count File:Line; Function
 ╎4 @Base/task.jl:411; (::IJulia.var"#15#18")()
 ╎ 4 @IJulia/src/eventloop.jl:8; eventloop(socket::ZMQ.Socket)
 ╎  4 @Base/essentials.jl:706; invokelatest
 ╎   4 @Base/essentials.jl:708; #invokelatest#2
 ╎    4 ...rc/execute_request.jl:67; execute_request(socket::ZMQ.Socke...
 ╎     4 ...c/SoftGlobalScope.jl:65; softscope_include_string(m::Modu...
 ╎    ╎ 4 @Base/loading.jl:1116; include_string(mapexpr::typeof...
3╎    ╎  4 @Base/boot.jl:360; eval
Total snapshots: 4
